In [1]:
#standard imports. Note: You must pip install nasdaqdatalink 1st
import os
import pandas as pd
import nasdaqdatalink
import requests
from pathlib import Path
import shutil
import sqlalchemy as db

In [2]:
# Linking my API key to .env in the same folder. The key is stored in the folder without any quotations around it 
# nasdaqdatalink.read_key(filename=".env")

# Create a temporary SQLite database and populate the database with content from the etf.db seed file
database_connection_string = 'sqlite:///county.db'

# Create an engine to interact with the SQLite database
engine = db.create_engine(database_connection_string)

# Confirm that table names contained in the SQLite database.
db.inspect(engine).get_table_names()



['county_coordinates', 'regions', 'zillow']

In [3]:
#A function to retrieve a dataframe of counties, zips, etc
def get_regions(regions):
    region_df=nasdaqdatalink.get_table('ZILLOW/REGIONS', region_type=regions)  
    return region_df

In [4]:
# Using get_regions to retrieve a list of counties
region_df = get_regions('county')
region_df[["county", "state"]] = region_df["region"].str.split(';', 1, expand=True)
region_df["state"] = region_df["state"].str.split(';', 1, expand=True)[0]

# Clean up
region_df["county"] = region_df["county"].str.replace(" County", "")

display(region_df.head())
display(region_df.tail())

# Import region_df into regions database table
# region_df.to_sql('regions', engine, index=False, if_exists='replace')

# db.inspect(engine).get_table_names()

,region_id,region_type,region,county,state
None,,,,,
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC
1,998,county,Duplin County; NC,Duplin,NC
2,997,county,Dubois County; IN; Jasper,Dubois,IN
3,995,county,Donley County; TX,Donley,TX
4,993,county,Dimmit County; TX,Dimmit,TX


,region_id,region_type,region,county,state
None,,,,,
2886,1003,county,Elmore County; AL; Montgomery,Elmore,AL
2887,1002,county,Elbert County; GA,Elbert,GA
2888,1001,county,Elbert County; CO; Denver-Aurora-Lakewood,Elbert,CO
2889,1000,county,Echols County; GA; Valdosta,Echols,GA
2890,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL


In [5]:
# Massaging zillow county names

# region_df['region']=region_df['region'].str.split(';', n=-1, expand=True)[0]

In [6]:
print(region_df)

     region_id region_type                                     region  county  \
None                                                                            
0          999      county      Durham County; NC; Durham-Chapel Hill  Durham   
1          998      county                          Duplin County; NC  Duplin   
2          997      county                  Dubois County; IN; Jasper  Dubois   
3          995      county                          Donley County; TX  Donley   
4          993      county                          Dimmit County; TX  Dimmit   
...        ...         ...                                        ...     ...   
2886      1003      county              Elmore County; AL; Montgomery  Elmore   
2887      1002      county                          Elbert County; GA  Elbert   
2888      1001      county  Elbert County; CO; Denver-Aurora-Lakewood  Elbert   
2889      1000      county                Echols County; GA; Valdosta  Echols   
2890       100      county  

In [7]:
#list(region_df['region_id'])

In [12]:
# The actual API call using the SDK.
# Instructions can be found here https://data.nasdaq.com/databases/ZILLOW/usage/quickstart/python
# Replace 'quandl' w/ 'nasdaqdatalink

# '''data = nasdaqdatalink.get_table('ZILLOW/DATA', paginate=True, indicator_id='ZSFH', region_id=list(region_df['region_id']))'''

data = nasdaqdatalink.export_table('ZILLOW/DATA', indicator_id='ZSFH', region_id=list(region_df['region_id']),filename='db.zip')

In [13]:
# Unzipping database from API call

shutil.unpack_archive('db.zip')

In [14]:
# Reading in Database
zillow_data=pd.read_csv(
    Path('ZILLOW_DATA_d5d2ff90eb7172dbde848ea36de12dfe.csv')
)

In [11]:
# Displaying database
display(zillow_data.head(10))
display(zillow_data.tail(10))

,indicator_id,region_id,date,value
0,ZSFH,100,2007-11-30,123760.0
1,ZSFH,100,2007-12-31,123754.0
2,ZSFH,100,2008-01-31,123605.0
3,ZSFH,100,2008-02-29,123393.0
4,ZSFH,100,2008-03-31,123095.0
5,ZSFH,100,2008-04-30,123018.0
6,ZSFH,100,2008-05-31,122855.0
7,ZSFH,100,2008-06-30,122883.0
8,ZSFH,100,2008-07-31,122853.0
9,ZSFH,100,2008-08-31,123015.0


,indicator_id,region_id,date,value
669306,ZSFH,999,2021-09-30,346041.0
669307,ZSFH,999,2021-10-31,355160.0
669308,ZSFH,999,2021-11-30,361432.0
669309,ZSFH,999,2021-12-31,368325.0
669310,ZSFH,999,2022-01-31,380061.0
669311,ZSFH,999,2022-02-28,390111.0
669312,ZSFH,999,2022-03-31,401621.0
669313,ZSFH,999,2022-04-30,411421.0
669314,ZSFH,999,2022-05-31,422028.0
669315,ZSFH,999,2022-06-30,430509.0


In [259]:
# Import region_df into regions database table
zillow_data.to_sql('zillow', engine, index=False, if_exists='replace')

db.inspect(engine).get_table_names()

['county_coordinates', 'regions', 'zillow']

In [260]:
# Read in county data with coordinates
county_coordinates_df=pd.read_csv(
    Path('counties_w_coordinates.csv')
)

county_coordinates_df.head()

,Sort [1],State,FIPS,County [2],County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,+32.536382°,–86.644490°
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",+30.659218°,–87.746067°
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,+31.870670°,–85.405456°
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°


In [287]:
region_df = get_regions('county')
region_df[["county", "state"]] = region_df["region"].str.split(';', 1, expand=True)
region_df["state"] = region_df["state"].str.split(';', 1, expand=True)[0]

# Clean up
region_df["county"] = region_df["county"].str.replace(" County", "")

display(region_df.head())
display(region_df.tail())

,region_id,region_type,region,county,state
None,,,,,
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC
1,998,county,Duplin County; NC,Duplin,NC
2,997,county,Dubois County; IN; Jasper,Dubois,IN
3,995,county,Donley County; TX,Donley,TX
4,993,county,Dimmit County; TX,Dimmit,TX


,region_id,region_type,region,county,state
None,,,,,
2886,1003,county,Elmore County; AL; Montgomery,Elmore,AL
2887,1002,county,Elbert County; GA,Elbert,GA
2888,1001,county,Elbert County; CO; Denver-Aurora-Lakewood,Elbert,CO
2889,1000,county,Echols County; GA; Valdosta,Echols,GA
2890,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL


In [288]:
##
##
## 
region_df['region_id']=region_df['region_id'].astype(int)

df2 = pd.merge(region_df, zillow_data, on=['region_id'])
df2.head()

,region_id,region_type,region,county,state,indicator_id,date,value
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-02-28,139430.0
1,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-03-31,139459.0
2,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-04-30,139659.0
3,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-05-31,139887.0
4,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-06-30,140303.0


In [289]:
# Column renaming
# data_county_df = data_county_df.rename(columns={"region" : "county"})
# data_county_df = data_county_df.rename(columns={"region" : "county"})
county_coordinates_df = county_coordinates_df.rename(columns={"County\xa0[2]" : "county"})
county_coordinates_df = county_coordinates_df.rename(columns={"region" : "region"})
county_coordinates_df = county_coordinates_df.rename(columns={"State" : "state"})

In [290]:
# display(df2.head())
county_coordinates_df.head()

,Sort [1],state,FIPS,county,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,+32.536382°,–86.644490°
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",+30.659218°,–87.746067°
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,+31.870670°,–85.405456°
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°


In [291]:
master_df = pd.merge(df2, county_coordinates_df, on=['county', 'state'])
master_df

,region_id,region_type,region,county,state,indicator_id,date,value,Sort [1],FIPS,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude


In [292]:
display(df2.head())
# county_coordinates_df=county_coordinates_df.iloc[1:, 1:]
display(county_coordinates_df.head())

,region_id,region_type,region,county,state,indicator_id,date,value
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-02-28,139430.0
1,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-03-31,139459.0
2,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-04-30,139659.0
3,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-05-31,139887.0
4,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-06-30,140303.0


,Sort [1],state,FIPS,county,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,NaN,NaN,NaN,NaN,NaN,-2010,km²,mi²,km²,mi²,km²,mi²,NaN,NaN
1,1.0,AL,1001.0,Autauga,Prattville,"54,571","1,539.58",594.436,25.776,9.952,"1,565.36",604.388,+32.536382°,–86.644490°
2,2.0,AL,1003.0,Baldwin,Bay Minette,"182,265","4,117.52","1,589.78","1,133.19",437.527,"5,250.71","2,027.31",+30.659218°,–87.746067°
3,3.0,AL,1005.0,Barbour,Clayton,"27,457","2,291.82",884.876,50.865,19.639,"2,342.68",904.515,+31.870670°,–85.405456°
4,4.0,AL,1007.0,Bibb,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°


In [293]:
# The secret sauce.   This removes the space by starting at index 1. 
df2['state']=df2['state'].str[1:]

In [295]:

pd.merge(df2, county_coordinates_df, on=['state', 'county'])

,region_id,region_type,region,county,state,indicator_id,date,value,Sort [1],FIPS,County Seat(s) [3],Population,Land Area,Land Area.1,Water Area,Water Area.1,Total Area,Total Area.1,Latitude,Longitude
0,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-02-28,139430.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
1,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-03-31,139459.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
2,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-04-30,139659.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
3,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-05-31,139887.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
4,999,county,Durham County; NC; Durham-Chapel Hill,Durham,NC,ZSFH,1997-06-30,140303.0,1922.0,37063.0,Durham,"267,587",740.673,285.975,30.798,11.891,771.471,297.866,+36.036589°,–78.877919°
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
628666,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-02-28,161462.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°
628667,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-03-31,162369.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°
628668,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-04-30,163859.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°
628669,100,county,Bibb County; AL; Birmingham-Hoover,Bibb,AL,ZSFH,2022-05-31,164684.0,4.0,1007.0,Centreville,"22,915","1,612.48",622.582,9.289,3.587,"1,621.77",626.169,+33.015893°,–87.127148°


In [166]:
query = """
SELECT * FROM regions;
"""

results = engine.execute(query)
list(results)

[('999', 'county', 'Durham County; NC; Durham-Chapel Hill', 'Durham', ' NC'),
 ('998', 'county', 'Duplin County; NC', 'Duplin', ' NC'),
 ('997', 'county', 'Dubois County; IN; Jasper', 'Dubois', ' IN'),
 ('995', 'county', 'Donley County; TX', 'Donley', ' TX'),
 ('993', 'county', 'Dimmit County; TX', 'Dimmit', ' TX'),
 ('992', 'county', 'Dillon County; SC', 'Dillon', ' SC'),
 ('991', 'county', 'Dickey County; ND', 'Dickey', ' ND'),
 ('990', 'county', 'Dewitt County; IL; Bloomington', 'Dewitt', ' IL'),
 ('99', 'county', 'Bent County; CO', 'Bent', ' CO'),
 ('989', 'county', 'Denver County; CO; Denver-Aurora-Lakewood', 'Denver', ' CO'),
 ('988', 'county', 'Denton County; TX; Dallas-Fort Worth-Arlington', 'Denton', ' TX'),
 ('986', 'county', 'Dekalb County; TN', 'Dekalb', ' TN'),
 ('985', 'county', 'DeKalb County; MO; St. Joseph', 'DeKalb', ' MO'),
 ('984', 'county', 'Dekalb County; GA; Atlanta-Sandy Springs-Roswell', 'Dekalb', ' GA'),
 ('983', 'county', 'DeSoto County; MS; Memphis', 'DeSoto

In [167]:
# Import county_coordinates_df into county_coordinates database table
county_coordinates_df.to_sql('county_coordinates', engine, index=False, if_exists='replace')

In [172]:
query = 'SELECT * FROM county_coordinates'

results = engine.execute(query)
# results.keys()
list(results)

[(None, None, None, None, None, '-2010', 'km²', 'mi²', 'km²', 'mi²', 'km²', 'mi²', None, None),
 (1.0, 'AL', 1001.0, 'Autauga', 'Prattville', '54,571', '1,539.58', '594.436', '25.776', '9.952', '1,565.36', '604.388', '+32.536382°', '–86.644490°'),
 (2.0, 'AL', 1003.0, 'Baldwin', 'Bay Minette', '182,265', '4,117.52', '1,589.78', '1,133.19', '437.527', '5,250.71', '2,027.31', '+30.659218°', '–87.746067°'),
 (3.0, 'AL', 1005.0, 'Barbour', 'Clayton', '27,457', '2,291.82', '884.876', '50.865', '19.639', '2,342.68', '904.515', '+31.870670°', '–85.405456°'),
 (4.0, 'AL', 1007.0, 'Bibb', 'Centreville', '22,915', '1,612.48', '622.582', '9.289', '3.587', '1,621.77', '626.169', '+33.015893°', '–87.127148°'),
 (5.0, 'AL', 1009.0, 'Blount', 'Oneonta', '57,322', '1,669.96', '644.776', '15.157', '5.852', '1,685.12', '650.628', '+33.977448°', '–86.567246°'),
 (6.0, 'AL', 1011.0, 'Bullock', 'Union Springs', '10,914', '1,613.06', '622.805', '6.057', '2.338', '1,619.11', '625.143', '+32.101759°', '–85.71

In [197]:
# Join region and county_coordinates by county
# query = """
#     SELECT 
#         *
#     FROM regions r
#     JOIN zillow z
#     ON r.region_id = z.region_id
#     """


query = """
    SELECT 
        count(*)
    FROM regions r
    JOIN zillow z
    ON r.region_id = z.region_id
    JOIN county_coordinates c
    ON r.county = c.'County\xa0[2]' AND r.state = c.state
    """

# query = """
#     SELECT 
#         *
#     FROM regions r
#     JOIN county_coordinates c
#     ON r.county = c.'County\xa0[2]'
#     """

# query = """
#     SELECT * 
#     FROM county_coordinates c
#     WHERE c.'County\xa0[2]' LIKE 'Dur%'
#     """

results = engine.execute(query)


list(results)

[(0,)]

In [184]:
query = """
    SELECT 
        count(*)
    FROM regions r
    JOIN zillow z
    ON r.region_id = z.region_id
    """

results = engine.execute(query)
list(results)

[(669316,)]

In [185]:
query = """
    SELECT 
        count(*)
    FROM regions r
    JOIN zillow z
    ON r.region_id = z.region_id
    """

results = engine.execute(query)
list(results)

[(669316,)]

In [189]:
query = """
    SELECT 
        county, state
    FROM regions r
    """

results = engine.execute(query)
list(results)

[('Durham', ' NC'),
 ('Duplin', ' NC'),
 ('Dubois', ' IN'),
 ('Donley', ' TX'),
 ('Dimmit', ' TX'),
 ('Dillon', ' SC'),
 ('Dickey', ' ND'),
 ('Dewitt', ' IL'),
 ('Bent', ' CO'),
 ('Denver', ' CO'),
 ('Denton', ' TX'),
 ('Dekalb', ' TN'),
 ('DeKalb', ' MO'),
 ('Dekalb', ' GA'),
 ('DeSoto', ' MS'),
 ('Dawson', ' TX'),
 ('Dawson', ' NE'),
 ('Dawson', ' MT'),
 ('Bell', ' TX'),
 ('Dawson', ' GA'),
 ('Dallas', ' TX'),
 ('Dallas', ' MO'),
 ('Dallas', ' IA'),
 ('Dallas', ' AR'),
 ('Dallas', ' AL'),
 ('Dallam', ' TX'),
 ('Dakota', ' NE'),
 ('Dakota', ' MN'),
 ('Custer', ' SD'),
 ('Bell', ' KY'),
 ('Custer', ' OK'),
 ('Custer', ' NE'),
 ('Custer', ' MT'),
 ('Custer', ' ID'),
 ('Custer', ' CO'),
 ('Cuming', ' NE'),
 ('Crosby', ' TX'),
 ('Craven', ' NC'),
 ('Cowley', ' KS'),
 ('Coweta', ' GA'),
 ('Bath', ' VA'),
 ('Cotton', ' OK'),
 ('Copiah', ' MS'),
 ('Cooper', ' MO'),
 ('Conway', ' AR'),
 ('Colusa', ' CA'),
 ('Collin', ' TX'),
 ('Colfax', ' NM'),
 ('Bath', ' KY'),
 ('Colfax', ' NE'),
 ('Coffey'